<h1> Sydney Metro Data Science </h1>


Import libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Download and Explore Dataset

In [36]:
import pandas as pd


 
df = pd.read_csv('postcodes.csv')


df.head()
#df.shape

    
    

,id,postcode,suburb,state,latitude,longitude
0,398,1001,Sydney,NSW,-33.79,151.27
1,399,1002,Sydney,NSW,-33.79,151.27
2,400,1003,Sydney,NSW,-33.79,151.27
3,401,1004,Sydney,NSW,-33.79,151.27
4,402,1005,Sydney,NSW,-33.79,151.27


Define a function to get GPS for a suburb

#### Data filtering

In [37]:
df=df[['postcode','suburb','longitude','latitude']]
df=df.drop_duplicates(subset=['suburb'], keep='first')
df = df[df.longitude != 0.00]
#df.head(10)
df.shape


(4592, 4)

In [43]:
address = 'Beecroft, NSW'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sydney are {}, {}.'.format(latitude, longitude))


/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Sydney are -33.7497631, 151.0657475.


In [39]:
df.head(10)

,postcode,suburb,longitude,latitude
0,1001,Sydney,151.27,-33.79
141,1208,Haymarket,151.66,-29.82
142,1209,Australia Square,151.18,-33.89
149,1216,Grosvenor Place,151.03,-33.74
154,1221,Royal Exchange,151.21,-33.87
159,1226,Queen Victoria Building,151.21,-33.87
164,1231,Sydney South,151.04,-33.82
183,1300,Darlinghurst,151.22,-33.88
208,1335,Potts Point,151.12,-33.82
209,1340,Kings Cross,148.02,-35.54


#### Create a map of Sydney with neighborhoods superimposed on top.

In [44]:
neighborhoods=df.head(100)
map_sydney = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, suburb in zip(neighborhoods['latitude'], neighborhoods['longitude'], neighborhoods['suburb']):
    label = '{}'.format(suburb)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sydney)  
    
map_sydney

#### Define Foursquare Credentials and Version

In [41]:
CLIENT_ID = 'VDFTNYUMJI3FIAI2E1GDVXWGMBJZYB30DCLYIJA5IIYQXX3F' # your Foursquare ID
CLIENT_SECRET = 'ZQSBPY4I2RWWPXV1ZHWCHV4YQORZPEOSWGFNDV33GWE2T4CO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VDFTNYUMJI3FIAI2E1GDVXWGMBJZYB30DCLYIJA5IIYQXX3F
CLIENT_SECRET:ZQSBPY4I2RWWPXV1ZHWCHV4YQORZPEOSWGFNDV33GWE2T4CO


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [67]:
#df_beecroft = df.loc[df['suburb'] == 'Beecroft']
#df['suburb'].iloc[0]

#df.query(suburb=='Beecroft')['longitude']

df.loc[df['suburb'] == 'Beecroft', 'longitude'].iloc[0]


151.06

In [62]:
df_beecroft['longitude']

611    151.06
Name: longitude, dtype: float64

Get the neighborhood's latitude and longitude values.

In [69]:
#neighborhood_latitude = df['latitude'].iloc[0] # neighborhood latitude value
#neighborhood_longitude = df['longitude'].iloc[0] # neighborhood longitude value
#neighborhood_name = df['suburb'].iloc[0] # neighborhood name

neighborhood_latitude = df.loc[df['suburb'] == 'Beecroft', 'latitude'].iloc[0] # neighborhood latitude value
neighborhood_longitude = df.loc[df['suburb'] == 'Beecroft', 'longitude'].iloc[0] # neighborhood longitude value



neighborhood_name = df.loc[df['suburb'] == 'Beecroft', 'suburb'].iloc[0] # neighborhood name


print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Beecroft are -33.75, 151.06.


#### Now, let's get the top 100 venues that are in Beecroft within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [70]:
# type your answer here
LIMIT = 100
radius = 500


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=VDFTNYUMJI3FIAI2E1GDVXWGMBJZYB30DCLYIJA5IIYQXX3F&client_secret=ZQSBPY4I2RWWPXV1ZHWCHV4YQORZPEOSWGFNDV33GWE2T4CO&v=20180605&ll=-33.75,151.06&radius=500&limit=100'

Send the GET request and examine the resutls

In [71]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bb5ba4e4434b9433f32a9c2'},
 'response': {'headerLocation': 'Beecroft',
  'headerFullLocation': 'Beecroft, Sydney',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': -33.7454999955, 'lng': 151.065402004367},
   'sw': {'lat': -33.7545000045, 'lng': 151.054597995633}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '59c9b11bf5e9d73dac18a3b0',
       'name': 'Woolworths',
       'location': {'address': '10 Hannah St',
        'crossStreet': 'Beecroft Rd',
        'lat': -33.74923225233269,
        'lng': 151.06508016586304,
        'labeledLatLngs': [{'label': 'display',
          'lat': -33.74923225233269,
          'lng': 151.06508016586304}],
        'distance': 477,
        'posta

let's borrow the get_category_type function from the Foursquare lab.

In [72]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [73]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Woolworths,Supermarket,-33.749232,151.065080
1,Yummy Kitchens,Chinese Restaurant,-33.750190,151.064923
2,Beecroft Place,Shopping Mall,-33.749210,151.064855
3,Cafe Bela,Café,-33.749362,151.065152
4,Beehive Cafe,Café,-33.750334,151.065210


And how many venues were returned by Foursquare?

In [74]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

7 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Beecroft

#### Let's create a function to repeat the same process to all the neighborhoods in Beecroft

In [75]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *beecroft_venues*.

In [82]:
neighborhoods = df[df['postcode'] == 2119].reset_index(drop=True)
neighborhoods.head()


beecroft_venues = getNearbyVenues(names=neighborhoods['suburb'],
                                   latitudes=neighborhoods['latitude'],
                                   longitudes=neighborhoods['longitude']
                                  )


Beecroft
Cheltenham


In [83]:
print(beecroft_venues.shape)
beecroft_venues.head()

(14, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Beecroft,-33.75,151.06,Woolworths,-33.749232,151.065080,Supermarket
1,Beecroft,-33.75,151.06,Yummy Kitchens,-33.750190,151.064923,Chinese Restaurant
2,Beecroft,-33.75,151.06,Beecroft Place,-33.749210,151.064855,Shopping Mall
3,Beecroft,-33.75,151.06,Cafe Bela,-33.749362,151.065152,Café
4,Beecroft,-33.75,151.06,Beehive Cafe,-33.750334,151.065210,Café


In [84]:
beecroft_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Beecroft,7,7,7,7,7,7
Cheltenham,7,7,7,7,7,7


In [85]:
print('There are {} uniques categories.'.format(len(beecroft_venues['Venue Category'].unique())))

There are 6 uniques categories.


In [88]:
# one hot encoding
beecrof_onehot = pd.get_dummies(beecroft_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
beecrof_onehot['Neighborhood'] = beecroft_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
beecrof_onehot = manhattan_onehot[fixed_columns]

beecrof_onehot

,Supermarket,Neighborhood,Café,Chinese Restaurant,Fast Food Restaurant,Italian Restaurant,Shopping Mall
0,1,Beecroft,0,0,0,0,0
1,0,Beecroft,0,1,0,0,0
2,0,Beecroft,0,0,0,0,1
3,0,Beecroft,1,0,0,0,0
4,0,Beecroft,1,0,0,0,0
5,0,Beecroft,0,0,0,1,0
6,0,Beecroft,0,0,1,0,0
7,1,Cheltenham,0,0,0,0,0
8,0,Cheltenham,0,1,0,0,0
9,0,Cheltenham,0,0,0,0,1
